In [58]:
# This file contains example of multi-research agents built using parallel workflows
# This workflow is used to execute tasks that are independent of each other, it is used for concurrent execution
# Importing libraries
import os
from dotenv import load_dotenv
from google.adk.agents import Agent, ParallelAgent, SequentialAgent
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search
from google.adk.runners import InMemoryRunner



In [59]:
# Check configuration
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print("API KEY FOUND")
else: 
    print("CONFIGURATION ERROR")

API KEY FOUND


In [60]:
def create_retry_config():
    '''This functions creates retry configuration'''
    return types.HttpRetryOptions(
        attempts=7,
        exp_base=2, # Used to trigger exponential backoff with 2^n
        initial_delay=2, # Initial delay of 2 secs before generating the response
        http_status_codes=[429, 500, 502, 503, 504]
    )

retry_config = create_retry_config()

In [61]:
# This is a multi-agent system that consists of 4 agents
# Tech researcher: The goal of this agent is to focus in AI/ML trends
tech_researcher=Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction='''Research the latest AI/ML trends. Include 3 key devlopments,
    the main companies involved in the research and the potential impact. 
    Keep the report concise upto 100 words''',
    tools=[google_search],
    output_key="tech_research"
)
print("tech_resercher agent created")

tech_resercher agent created


In [62]:
# Medical researcher: The goal of this agent is to focus of trends related to medical field
health_researcher=Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction='''Research the latest breakthrough in the medical field.
    Include recent 3 advancements, their practical use case and estimated timelines''',
    tools=[google_search],
    output_key="health_research"
)
print("Health researcher created")

Health researcher created


In [63]:
# Finance researcher: The goal is to find latest trends in the fintech world
finance_researcher=Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction='''Research the latest breakthrough in the fintech field.
    Include recent 3 discoveries, their practical use case and estimated timelines''',
    tools=[google_search],
    output_key="finance_research"
)
print("Finance researcher created")
print(type(finance_researcher))

Finance researcher created
<class 'google.adk.agents.llm_agent.LlmAgent'>


In [64]:
# This agent is used to synthesize the output generated from the other agent, and present it carefully to the user
aggregator_agent=Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction='''Combine the research findings in a single executive memory.
    **Technology Trends:**,
    {tech_research}

    **Health breakthrough:**
    {health_research}

    **Fintech Innovations:**
    {finance_research}

    Your summary should highlight common themes, surprising connections and the most important key takeaway''', 
    output_key="executive_summary"
)
print("Aggregator agent created")
print(type(aggregator_agent))

Aggregator agent created
<class 'google.adk.agents.llm_agent.LlmAgent'>


In [65]:
# Building Parallel agent framework

parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("Parallel and Sequential agents created")

Parallel and Sequential agents created


In [66]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, Finance"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/arpit/devlopement/agentic-ai-google-kaggle/myvenv/lib/python3.9/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, Finance
TechResearcher > **Key AI/ML Developments in 2025:**

1.  **Generative AI Expansion:** Beyond text and images, Generative AI is now creating novel proteins, materials, and complex multimedia content (video, music). This is revolutionizing scientific discovery and industrial applications.
2.  **Agentic AI and Autonomous Systems:** AI systems are gaining the ability to act autonomously, perceive, reason, and make decisions with minimal human input, transforming fields like healthcare, logistics, and finance.
3.  **Explainable AI (XAI) and Ethical Frameworks:** As AI becomes more integrated into critical sectors, there's a growing emphasis on XAI to understand AI decision-making, ensure fairness, and build trust.

**Main Companies Involved:** OpenAI, Google (DeepMind), Microsoft, NVIDIA, Anthropic, Amazon (AWS), Meta, IBM, Apple, and Databricks are leading research and development